In [33]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# this loads and starts pyterrier so that it also works in the TIRA
ensure_pyterrier_is_loaded()

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt

In [34]:
import spacy


nlp = spacy.load("en_core_web_sm")
spacy_stopwords = set(nlp.Defaults.stop_words)


!rm -Rf /tmp/index
file_path = "stopwords.txt"

with open(file_path, 'w+') as file:
    for element in spacy_stopwords:
        file.write(element+ "\n")

pt.set_property('stopwords.filename','./stopwords.txt')

In [35]:
data = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
print('See the first two queries:')
topics = data.get_topics('title')
print(topics.head(2))

See the first two queries:
  qid                                     query
0   1  retrieval system improving effectiveness
1   2  machine learning language identification


In [36]:
print('Build index:')
indexer = pt.IterDictIndexer("/tmp/index", overwrite = True, blocks = True,meta = {'docno':100, 'text': 20480}, stemmer = 'PorterStemmer')
!rm -Rf /tmp/index
index_ref = indexer.index(data.get_corpus_iter())

print('Done. Index is created')

Build index:


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  65%|██████▌   | 83097/126958 [00:16<00:09, 4741.14it/s]

18:32:43.709 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1980.sigirconf_conference-80.24) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:23<00:00, 5292.70it/s] 


18:32:53.452 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 4 empty documents
Done. Index is created


In [37]:
index = pt.IndexFactory.of(index_ref)

bm25 = pt.BatchRetrieve(index, wmodel="BM25", verbose=True)
sdm = pt.rewrite.SequentialDependence()
pl2 = pt.BatchRetrieve(index, wmodel="PL2", verbose=True)

In [38]:
#Query Expansion
bo1 = pt.rewrite.Bo1QueryExpansion(index) 

#Pipeline
pipe = (bm25 % 100) >> bo1 >> sdm >> pl2

In [39]:
print('Create run')

run = pipe(topics)

print('Done, run was created')

Create run


BR(BM25):   0%|          | 0/68 [00:00<?, ?q/s]

BR(PL2): 100%|██████████| 68/68 [00:16<00:00,  4.21q/s]

Done, run was created


In [40]:
persist_and_normalize_run(run, 'retrieval_system', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
